This function computes the interpolant for the Lagrangian linear momentum barrier equation:

\begin{equation}
 \mathbf{x}_0'=\nu \rho \mathbf{J} \mathbf{\nabla_0 \overline{\hat{\omega} (\mathbf{F_{t_0}^{t}(x_0)}, t)}}, \label{eq: linearmomentum} \tag{1}
\end{equation}
 
where $ \nu $ is the viscosity, $ \rho $ the density of the fluid, $ \mathbf{J} = \begin{pmatrix} 0 && 1 \\ -1 && 0 \end{pmatrix} $ and $ \hat{\omega} $ is the scalar valued vorticity in 2D. $ \overline{(\ldots)} $ denotes a temporal average.
 
 For simplicity we can normalize equation $ \ref{eq: linearmomentum} $ by $ \nu $ and $ \rho $ and thus simply omit them for simplicity as they will play no role when exploring the structures of the field $ \mathbf{x'} $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X_domain | array (Ny, Nx)| X-meshgrid of initial conditions |
| Y_domain | array (Ny, Nx)| Y-meshgrid with Ny=Nx=1024 over which $ \hat{\omega}(\mathbf{x}, t) $ is defined |
| X | array (1024, 1024)| X-meshgrid over which $ \hat{\omega}(\mathbf{x}, t) $ is defined |
| Y | array (1024, 1024)| Y-meshgrid over which $ \hat{\omega}(\mathbf{x}, t) $ is defined |
| omega | array (1024, 1025, 251) | $ \hat{\omega}(\mathbf{x}, t) $ |
| Fmap | array (Ny, Nx, N) | $ \mathbf{F}_{t_0}^{t}(\mathbf{x}_0) $, with $ t \in [t_0, t_N] $  |
| time | array (N, ) | $ t \in [t_0, t_N] $ |
| time_data | array | time of the velocity data |
| aux_grid | list (2,) | aux_grid[0]: dx_auxiliary spacing <br /> aux_grid[1]: dy_auxiliary spacing |
| Interpolant | list (2,) | Interpolant[0]: Interpolant for x-component of $ \mathbf{x_0}' $ <br /> Interpolant[1]: Interpolant for y-component of $ \mathbf{x_0}' $ |

In [1]:
# import Rectangular bivariate spline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

# import package for progress bar
from tqdm.notebook import tqdm

# Import numpy 
import numpy as np

In [2]:
def LagrangianActiveMomentum(X_domain, Y_domain, X, Y, omega, Fmap, time, time_data, aux_grid):
    '''
    Compute interpolant of right-hand-side of Lagrangian momentum barrier equation.
    
    Parameters:
        X_domain: array(Ny, Nx), X-meshgrid of domain
        Y_domain: array(Ny, Nx), Y-meshgrid of domain
        X:        array(NY, NX), X-meshgrid of data domain
        Y:        array(NY, NX), Y-meshgrid of data domain
        omega:    array(NY, NX, NT), 3D meshgrid of vorticity over spatial and temporal data domain
        Fmap:     array(N, 2, Ny*Nx), Flow map of trajectories
        time:     array(N, ), time
        time_data: array(1,NT), time data
        aux_grid: list(2,), specifies spacing of x/y auxiliary grid components
        
    Returns:
        Interpolant_Jgrad_delta_omega: list(2,) of Interpolant-objects for right-hand-side of Lagrangian vorticity.
        Interpolant_Jgrad_delta_omega[0] --> Interpolant for x-component
        Interpolant_Jgrad_delta_omega[1] --> Interpolant for y-component
    '''
    
    # auxiliary grid
    rho_x = aux_grid[0]
    rho_y = aux_grid[1]
    
    # iterate over all trajectories and compute omega along them
    
    # compute interpolant for omega over meshgrid X, Y over time
    Interpolant_omega = []
    for i in tqdm(range(omega.shape[2])):
        Interpolant_omega.append(RBS(Y[:,0], X[0,:], omega[:,:,i]))

    Omega = np.zeros((Fmap.shape[0], Fmap.shape[2]))
    
    # evaluate omega at 'x' at time 't' 
    # iterate over time:
    for c, i in enumerate(time[:-1]):
        
        dt_data = time_data[0,1]-time_data[0,0]
        k = np.searchsorted(time_data[0,:], i)
            
        Omegai = Interpolant_omega[k](Fmap[c,1,:].ravel(), Fmap[c,0,:].ravel(), grid = False)
        Omegaf = Interpolant_omega[k+1](Fmap[c,1,:].ravel(), Fmap[c,0,:].ravel(), grid = False)
        Omega[c,:] = (time_data[0,k+1]-i)/dt_data*Omegai + (i-time_data[0,k])/dt_data*Omegaf

    Omega_avg = np.mean(Omega, axis = 0)
    Omega_avg = Omega_avg.reshape((X_domain.shape[0], Y_domain.shape[1]))
    
    # Interpolant for average of omega
    Interpolant_omega_avg = RBS(Y_domain[:,0], X_domain[0,:], Omega_avg)
    
    Jgrad_omega_avg = np.zeros((X_domain.shape[0], Y_domain.shape[1], 2))*np.nan
        
    # compute x'=J grad(average(omega)), with J = [[0, 1], [-1, 0]]
    for i in range(X_domain.shape[0]):
        
        for j in range(Y_domain.shape[1]):
            
            # apply periodic boundary conditions to point 'x'
            x = np.array([X_domain[i, j], Y_domain[i, j]])
            
            # evaluate grad(omega) at 'x' using auxiliary grid
            xR = (x[0] + rho_x)%(X[0,-1]-X[0,0])
            OmegaR = Interpolant_omega_avg(x[1], xR)[0][0]
            
            xL = (x[0] - rho_x)%(X[0,-1]-X[0,0])
            OmegaL = Interpolant_omega_avg(x[1], xL)[0][0]
            
            xU = (x[1] + rho_y)%(Y[-1,0]-Y[0,0])
            OmegaU = Interpolant_omega_avg(xU, x[0])[0][0]
            
            xD = (x[1] - rho_y)%(Y[-1,0]-Y[0,0])
            OmegaD = Interpolant_omega_avg(xD, x[0])[0][0]
            
            Jgrad_omega_avg[i, j, 0] = -(OmegaU - OmegaD)/(2*rho_y)
            Jgrad_omega_avg[i, j, 1] = (OmegaR - OmegaL)/(2*rho_x)
    
    Interpolant = []
    Interpolant.append(RBS(Y_domain[:,0], X_domain[0,:], Jgrad_omega_avg[:,:,0]))
    Interpolant.append(RBS(Y_domain[:,0], X_domain[0,:], Jgrad_omega_avg[:,:,1]))
    
    return Interpolant